**VGG 이용한 Inference 구현**

In [3]:
import numpy as np
import json
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision import models, transforms

net=models.vgg16(pretrained=True)
net.eval()

print(net)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 97.1MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
class BaseTransform():
  def __init__(self, resize, mean, std):
    self.base_transform=transforms.Compose([
        transforms.Resize(resize),
        transforms.CenterCrop(resize),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

  def __call__(self, img):
    return self.base_transform(img)

In [ ]:
ILSVRC_class_index=json.load(open('./data/imagenet_class_index.json', 'r'))
ILSVRC_class_index

In [ ]:
class ILSVRCPredictor():
  def __init__(self, class_index):
    self.class_index=class_index

  def predict_max(self, out):
    maxid=np.argmax(out.detach().numpy())
    predicted_label_name=self.class_index[str(maxid)][1]

    return predicted_label_name

In [ ]:
predictor=ILSVRCPredictor(ILSVRC_class_index)

image_file_path='./data/goldenretriever-3724972_640.jpg'
img=Image.open(image_file_path)

resize=224
mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)

transform=BaseTransform(resize, mean, std)
img_transformed=transform(img)
inputs=img_transformed.unsqueeze_(0)

out=net(inputs)
result=predictor.predict_max(out)

print(result)

**VGG 이용한 전이학습 구현**

전이학습이란: 학습된 모델을 기반으로 최종 출력층을 바꿔 학습하는 기법이다.

In [ ]:
class ImageTransform():
  def __init__(self, resize, mean, std):
    self.data_transform={
        'train': transforms.Compose([
            transforms.RandomResizedCrop(
                resize, scale=(0.5, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
            ]),
         'val': transforms.Compose([
            transforms.Resize(resize),
            transforms.CenterCrop(resize),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
            ]),
    }

  def __call__(self,img, phase='train'):
    return self.data_transform[phase](img)

In [ ]:
import os.path as osp
import glob
def make_datapath_list(phase='train'):
  rootpath="./data/hymenoptera_data"
  target_path=osp.join(rootpath+phase+'/**/*.jpg')
  print(target_path)

  path_list=[]

  for path in glob.glob(target_path):
    path_list.append(path)

  return path_list

train_list=make_datapath_list(phase='train')
val_list=make_datapath_list(phase='val')

train_list

In [ ]:
from torch.utils.data import Dataset, DataLoader

class HymenopteraDataset(Dataset):

  def __init__(self, file_list, transform=None, phase='train'):
    self.file_list = file_list   # 파일 경로 리스트
    self.transform = transform   # 전처리 클래스의 인스턴스
    self.phase = phase   # train or val 지정

def __len__(self):
    return len(self.file_list)

def __getitem__(self, index):
    # index번째의 화상 로드
    img_path = self.file_list[index]
    img = Image.open(img_path)   # [높이][폭][색RGB]

    # 화상의 전처리 실시
    img_transformed = self.transform(
        img, self.phase) # torch.Size([3, 224, 224])

    # 화상 라벨을 파일 이름에서 추출
    if self.phase == "train":
        label = img_path[30:34]
    elif self.phase == "val":
        label = img_path[28:32]

    # 라벨을 숫자로 변경
    if label == "ants":
        label = 0
    elif label == "bees":
        label = 1

    return img_transformed, label


In [ ]:
train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(resize, mean, std), phase='train')

val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(resize, mean, std), phase='val')


In [ ]:
# 미니 배치 크기 지정
batch_size = 32

# 데이터 로더 작성
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 사전형 변수에 정리
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

# 동작 확인
batch_iterator = iter(dataloaders_dict["train"])  # 반복자(iterator)로 변환
inputs, labels = next(batch_iterator)  # 첫 번째 요소 추출
print(inputs.size())
print(labels)

In [ ]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

params_to_update = []

# 학습시킬 파라미터명
update_param_names = ["classifier.6.weight", "classifier.6.bias"]

# 학습시킬 파라미터 외에는 동결 상태로 설정하여 값이 변하지 않도록 설정
for name, param in net.named_parameters():
    if name in update_param_names:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False

In [ ]:
import tqdm
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # 에포크 루프
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # 학습 및 검증 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 모델을 훈련 모드로
            else:
                net.eval()   # 모델을 평가 모드로

            epoch_loss = 0.0  # 에포크 손실 합
            epoch_corrects = 0  # 에포크 정확한 수

            # 학습과정의 정확성 및 검증 성능을 확인하기 위해서 epoch=0일 때의 훈련 생략
            if (epoch == 0) and (phase == 'train'):
                continue

            # 데이터 로더로 미니 배치를 꺼내는 루프
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # 옵티마이저 초기화
                optimizer.zero_grad()

                # 순전파 계산
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 손실 계산
                    _, preds = torch.max(outputs, 1)  # 최댓값 예측

                    # 훈련 시에만 오차 역전파
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 손실 합계 갱신
                epoch_loss += loss.item() * inputs.size(0)
                # 정답 수의 합계 갱신
                epoch_corrects += torch.sum(preds == labels.data)
                            # 에폭당 손실과 정답률 표시
                epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
                epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc))

import torch.optim as optim
# 최적화 기법 설정
optimizer = optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)

# 학습 및 검증 실시
num_epochs=2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

